In [ ]:
# %load DGforStoke0715.py
#from dolfin import *

from fenics import *

def E(u):
    return 0.5*( grad(u) + grad(u).T )
def tau(u,n):
    return u - inner(u,n)*n

# Define parameters
alpha = 4.0
gamma = 8.0
nu = 1.0
mu =1.0
theta = 1.0 #SIP
Kf = 2 #degree

# FIXME: Make mesh ghosted
#parameters["ghost_mode"] = "shared_facet"

# Define class marking Dirichlet boundary (x = 0 or x = 1)
class DirichletBoundary(SubDomain):
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0]*(1 - x[0]), 0)

# Define class marking Neumann boundary (y = 0 or y = 1)
class NeumanBoundary(SubDomain):
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1]*(1 - x[1]), 0)
    
# Create mesh and define function space
#nu = 1
mesh = UnitSquareMesh(48, 48)
P1dv = VectorElement("DG", triangle, 2); P1d = FiniteElement("DG", triangle, 2)
P1dP1d = MixedElement([P1dv, P1d]); W = FunctionSpace(mesh, P1dP1d)
P0 = FiniteElement("DG", triangle, 0); Q0 = FunctionSpace(mesh, P0)

# Define test and trial functions
(v,q) = TestFunctions(W)
(u,p) = TrialFunctions(W)


# Define normal vector and mesh size
n = FacetNormal(mesh)
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2

# Define the source term f, Dirichlet term u0 and Neumann term g

uSex = Expression(('-exp(x[0])*(x[1]*cos(x[1])+sin(x[1]))', \
               'exp(x[0])*x[1]*sin(x[1])'),degree=5)
pSex = Expression('exp(x[0])*sin(x[1])', degree=5)

#f = Expression(('-exp(x[0])*sin(x[1])', \
#               '-exp(x[0])*cos(x[1])'),degree=3)
f = Expression(('0','0'),degree=3)

# Mark facets of the mesh
boundaries = MeshFunction('size_t', mesh, 0)
NeumanBoundary().mark(boundaries, 2)
DirichletBoundary().mark(boundaries, 1)

# Define outer surface measure aware of Dirichlet and Neumann boundaries
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

In [2]:
# Define variational problem

#Ah(uv)
Ahuv = \
mu*inner(E(u),E(v))*dx \
-mu*inner(avg(E(u))*n('+'),jump(v))*dS - mu*inner(E(u)*n,v)*ds \
+theta*mu*inner(avg(E(v))*n('+'),jump(u))*dS +theta*mu*inner(E(v)*n,u)*ds \
+gamma/h_avg*inner(jump(u),jump(v))*dS + gamma/h*inner(u,v)*ds \


#Bh(vp)
Bhvp= - p*div(v)*dx + avg(p)*inner(jump(v),n('+'))*dS + p*inner(v,n)*ds

#Bh(uq)
Bhuq = - q*div(u)*dx + avg(q)*inner(jump(u),n('+'))*dS + q*inner(u,n)*ds

# a_Stokes
a = Ahuv + Bhvp - Bhuq

# RHS
#inner(fnew,v)*dx \
#L =  inner(-div(nu*E(uSex))+grad(pSex),v)*dx  \
#L =  inner(-div(nu* E(Constant(1.0, cell = mesh.ufl_cell()) *uSex))\
#+ grad(Constant(1.0, cell = mesh.ufl_cell()) *pSex),v)*dx\

L=inner(f,v)*dx \
+ mu*inner(E(v)*n,uSex)*ds \
- q*inner(uSex,n)*ds \
+ gamma/h*inner(uSex,v)*ds #

# Solve problem
w = Function(W)
solve(a == L, w)
(u,p) = w.split()

# additive const
p0 = p((0,0)); print ("p0 =", p0); p = p-p0

# error computation
V = FunctionSpace(mesh, P1dv); Q = FunctionSpace(mesh, P1d);
uSer = project(u-uSex, V); pSer = project(p-pSex, Q)
uS_L2er = sqrt( assemble(inner(uSer,uSer)*dx) ); uS_H1er = sqrt( assemble( inner(grad(uSer),grad(uSer))*dx ) ); pS_L2er = sqrt( assemble(pSer*pSer*dx) )
print ("\n")
print ("|u-uh|_{L2,S} \t |u-uh|_{H1,S} \t |p-ph|_{L2,S}")
print ("%g \t %f \t %f" % (uS_L2er, uS_H1er, pS_L2er))
print ("\n")

# h_avg =avg(h)
# This is v^+ - v^-
#jump(v)
# This is (v^+ - v^-) n
#jump(v, n)


Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
p0 = -1827416.89265
Calling FFC just-in-time (JIT) compiler, this may take some time.


|u-uh|_{L2,S} 	 |u-uh|_{H1,S} 	 |p-ph|_{L2,S}
4.31706e-07 	 0.000102 	 0.001052


